In [8]:
# -*- coding: utf-8 -*-
#审查
import xml.etree.cElementTree as ET
import pprint
from collections import defaultdict as dfdict
import re
from pypinyin import pinyin, lazy_pinyin
zhPattern = re.compile(u'[\u4e00-\u9fa5]+')
def count_tags(filename):
    count = dfdict(int)
    for node in ET.iterparse(filename):
       # print node[1].tag
        count[node[1].tag]+=1

    return count

In [9]:
#检查‘k’是否有问题
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

def key_type(element, keys):
    if element.tag == "tag":

        if lower.search(element.attrib['k'])!=None:
            keys['lower'] +=1

        elif lower_colon.search(element.attrib['k'])!=None:           
            keys['lower_colon'] +=1

        elif problemchars.search(element.attrib['k'])!=None:            
            keys['problemchars'] +=1
        else:
            keys['other']+=1    
    return keys

def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)    
    return keys

In [26]:
#有冒号的有17651个，说明需要对k进行冒号值进行调整
#other 有137个也需要看看

In [10]:
street_type_re =re.compile(r'\b\S+\.?$', re.IGNORECASE)
street_types = dfdict(set)

expected=["Street", "Avenue", "Boulevard", "Drive", "Court", "Ct", "Place", "Square", "Lane", "Road",
            "Trail", "Parkway", "Commons", "Park", "Broadway", "Circle", "Highway", "Trail",
            "Way", "West", "North", "Terrace", "Plaza", "Market"]
mapping = { "St": "Street","St.": "Street","ST": "Street","street":"Street","st": "Street",'Jie':'Street',
            'jie':'Street',
            "Rd": "Road","raod":"Road","road": "Road","Lu":'Road',
            "Ln":"Lane",
            "BLVD": "Boulevard",
            "Acenue": "Avenue", "Ave": "Avenue","avenue": "Avenue", "Av": "Avenue",
            "Hwy": "Highway",
            "Blvd": "Boulevard",
            "Ct": "Court",
            "E": "East","S": "South","W": "West","N": "North","S.": "South",
            "NE": "Northeast","NW": "Northwest","SE": "Southeast","SW": "Southwest",
            "Dadao": "DaDao"
            }
def audit_street_type(street_types, street_name):

    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)
    
#def print_sorted_dict(d):
#    keys = d.keys()
#    keys = sorted(keys, key=lambda s:s.lower())
#    for k in keys:
#        v =d[k]
#        print "%s:%d"%(k,v)
            

def is_street_name(elem):

    return elem.attrib['k'] == "addr:street"
#    return elem.attrib['k'] == "name:"

def audit(osm_file):
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag =="way" or elem.tag ==  "node":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                     audit_street_type(street_types, tag.attrib['v'])
    pprint.pprint(dict(street_types))
    osm_file.close()    
    return street_types 


def update_name(name, mapping):
#    for key in mapping:
#        keyy=key
##        a=name.find(keyy)
#        if a> 0  :                           
##            b=key                
#            b =mapping[b]
#            name =name[:a]+b
    name = Chinese_English(name)
    name = name.split()
    for i in range(len(name)):
        if name[i] in mapping:
            name[i] = mapping[name[i]]
    name = " ".join(name)  #http://blog.csdn.net/chixujohnny/article/details/53301995
    return name 
#识别是不是中文
def Chinese_English(name): #https://zhidao.baidu.com/question/1958984257226168500.html
    match = zhPattern.search(name)
    if match:
        return ''.join(lazy_pinyin(name))
    else:
        return name

osm_file=open("/Users/chebyshev/Downloads/THmap","r")
filename="/Users/chebyshev/Downloads/THmap"


count =count_tags(filename)
pprint.pprint(count)
keys = process_map('/Users/chebyshev/Downloads/THmap')
pprint.pprint(keys)

print '__________________________________'
street_types = audit(osm_file)
pprint.pprint(street_types)
print '__________________________________'
for street_types, ways in street_types.items():
    for name in ways:
        print name
        better_name = update_name(name, mapping)
        print (name, '=>', better_name)

defaultdict(<type 'int'>, {'node': 340581, 'member': 121970, 'nd': 392909, 'tag': 137940, 'bounds': 1, 'note': 1, 'meta': 1, 'relation': 1790, 'way': 41245, 'osm': 1})
{'lower': 120152, 'lower_colon': 17651, 'other': 137, 'problemchars': 0}
__________________________________
{u'168\u53f7': set([u'\u5e7f\u5dde\u5e02\u8d8a\u79c0\u533a\u5317\u7ad9\u8def168\u53f7']),
 '39': set(['Hong Road No 39']),
 u'510610': set([u'China, Guangdong Sheng, Guangzhou Shi, Tianhe Qu, TianHe GongYuan, Tianhe Rd, \u592a\u53e4\u6c47L307\u53f7, \u90ae\u653f\u7f16\u7801: 510610']),
 u'531\u53f7\u5927\u9662': set([u'\u6e2f\u524d\u8def531\u53f7\u5927\u9662']),
 u'58\u4e0060': set([u'\u5e7f\u5dde\u5e02\u8354\u6e7e\u533a\u897f\u6751\u8857\u9053\u529e\u4e8b\u5f6d\u57ce\u4e1c\u8def58\u4e0060']),
 'Bilu': set(['Lang wang Bilu']),
 u'East': set([u'\u73e0\u6c5f\u4e1c\u8def Zhujiang Road East']),
 'Guangdong,': set(['Room 322, Jintao Building,26# Guangyuanzhong Road, Baiyun District,Guangzhou, Guangdong,']),
 'Guangzhou'